In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Read data
df_1m = pd.read_parquet("../data/silver/power_load_1m.parquet")
df_5m = pd.read_parquet("../data/silver/power_load_5m.parquet")
df_10m = pd.read_parquet("../data/silver/power_load_10m.parquet")

In [5]:
def plot_load_data(df, title_suffix):
    # -----------------------------
    # Ensure types
    # -----------------------------
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="raise")
    df = df.sort_values("timestamp")

    # -----------------------------
    # Create day label + seconds-of-day
    # -----------------------------
    df["date"] = df["timestamp"].dt.normalize()  # midnight of that date
    df["second_of_day"] = (df["timestamp"] - df["date"]).dt.total_seconds().astype(int)

    # -----------------------------
    # Select which days to plot (first 6 by date)
    # -----------------------------
    unique_dates = df["date"].drop_duplicates().sort_values()
    selected_dates = unique_dates.iloc[:6]   # like MATLAB [1:6]
    resolution = 1

    fig = go.Figure()

    for d in selected_dates:
        day_df = df[df["date"] == d].sort_values("second_of_day")
        day_df = day_df.iloc[::resolution]

        fig.add_trace(
            go.Scatter(
                x=day_df["second_of_day"].to_numpy() / 3600,
                y=day_df["avg_load"].to_numpy() / 1e3,
                mode="lines",
                name=str(d.date()),
            )
        )

    fig.update_layout(
        title=f"Daily power/demand data over {len(selected_dates)} day(s) - {title_suffix}",
        xaxis_title=f"time [hours in {resolution} sec. resolution]",
        yaxis_title="power/demand [kW]",
        template="plotly_white",
    )

    fig.show()

plot_load_data(df_1m, "1-minute resolution")
plot_load_data(df_5m, "5-minute resolution")
plot_load_data(df_10m, "10-minute resolution")